In [16]:
import os 

# Run global_variables, synthesize_functions and mlp_model
%run ~/violin-renderer/src/global_variables.ipynb
%run ~/violin-renderer/src/models/lstm/lstm_model.ipynb
%run ~/violin-renderer/src/synthesize/synthesize_functions.ipynb
%run ~/violin-renderer/src/models/lstm/data_processing.ipynb

Skip downloading as the MuseScore General soundfont is found.


In [17]:
testing_X, testing_y = load_testing_data()

In [18]:
def trim_sequence(input_song, true_song):
    all_source_inputs = []
    all_ground_truths = []

    for note_source, note_truth in zip(input_song, true_song):
        if note_source[1] > 64 * 24:
            break
            
        note_source = list(note_source)
        note_truth = list(note_truth)
        
        note_source[0] = scale_timings(note_source[0])
        note_source[1] = scale_timings(note_source[1])
        note_source[2] = scale_pitch(note_source[2])
        
        all_source_inputs.append(note_source)
        all_ground_truths.append(note_truth)

    return all_source_inputs, all_ground_truths

In [19]:
# Create an output directory in /src, generate all results from the test dataset and synthesize all of them
# Setting up paths and output folders
SOURCE_INPUT_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/source-input'
TEMPO_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/tempos'
OUTPUT_PATH = HOME_PATH + '/src/lstm-output'
composer_directories = os.listdir(SOURCE_INPUT_PATH)

In [22]:
input_dim = 3
hidden_dim = 32
layer_dim = 2
output_dim = 2

# initialize the MLP
model = LSTM(input_dim, hidden_dim, layer_dim, output_dim)

model.load_state_dict(torch.load(HOME_PATH + '/src/models/lstm/lstm.pt'))

<All keys matched successfully>

In [23]:
# Get generated output values from all songs
results = {}

for input_path, true_path in zip(testing_X, testing_y):
    trimmed_notes = []
    trimmed_notes, trimmed_truth = trim_sequence(testing_X[input_path], testing_y[true_path])

    popped_values = []
    
    trimmed_notes = [trimmed_notes]
    trimmed_notes = torch.Tensor(trimmed_notes)
    length = [len(trimmed_notes[0])]
    length = torch.Tensor(length)

    pred = model(trimmed_notes, length)

    print(pred)
    
    pred = pred.tolist()

    # # remove nonsensical values
    # for index in range(len(pred)):
    #     if pred[0][index][1] < pred[0][index][0]:
    #         popped_values.append(index)

    # for index in reversed(popped_values):
    #     pred.pop(index)
        
    results[input_path] = pred

tensor([[[ 9.9444, 10.3869],
         [ 1.7208,  2.1410],
         [ 2.5071,  2.9051],
         [ 2.3087,  2.7340],
         [ 2.8701,  3.3128],
         [ 3.1495,  3.6094],
         [ 3.5029,  3.9778],
         [ 3.7400,  4.2289],
         [ 3.9458,  4.4487],
         [ 4.0808,  4.5971],
         [ 4.2644,  4.7884],
         [ 5.3701,  5.8537],
         [ 7.0280,  7.3974],
         [ 7.1291,  7.4749],
         [ 7.0394,  7.3902],
         [ 7.0939,  7.4493],
         [ 7.2079,  7.5676],
         [ 7.3314,  7.6956],
         [ 7.4560,  7.8249],
         [ 7.5872,  7.9609],
         [ 7.7417,  8.1204],
         [ 7.9302,  8.3144],
         [ 8.1738,  8.5644],
         [ 8.4891,  8.8871],
         [ 8.8897,  9.2966],
         [ 9.3716,  9.7889],
         [ 9.9031, 10.3318],
         [10.4127, 10.8527],
         [10.8304, 11.2803],
         [11.1276, 11.5855],
         [11.3221, 11.7862],
         [11.4496, 11.9181],
         [11.5375, 12.0092],
         [11.6057, 12.0792],
         [11.6

In [24]:
# Synthesize
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

for notes_path, generated_results in results.items():
    name = '/' + notes_path.split('/')[-1]
    name = name.replace('.csv', '')
    generated_timings = [(note[0], note[1]) for note in generated_results[0]]
    synthesize_generated_output(notes_path, generated_timings, OUTPUT_PATH, name)

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/wanninglu/violin-renderer/src/lstm-output/shunske-sato_bwv1005_mov2.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/wanninglu/violin-renderer/src/lstm-output/shunske-sato_bwv1001_mov1.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/wanninglu/violin-renderer/src/lstm-output/iris-kengen_bwv1001_mov2.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
Sou